# Procédures Douanières
## Système Intelligent d'Automatisation des Procédures Douanières France-Algérie

**Projet de Fin de Master**

---

## Critères d'Évaluation

| Critère | Poids | Notre Réponse |
|---------|-------|---------------|
| **Qualité du Code** | 25% | Architecture modulaire, patterns professionnels, code maintenable |
| **Maîtrise Technique** | 25% | Intelligence artificielle avancée (LLM, Vision, Tool-Calling) |
| **Frontend/Visualisation** | 25% | Interface moderne et réactive |
| **Exécution du Cas d'Usage** | 25% | Workflow complet : classification → génération documents |

---
# 1. Contexte et Problématique
---

## Le Problème

Le commerce international entre la France et l'Algérie implique des **procédures douanières complexes** :

1. **Classification des produits** : Trouver le bon code TARIC parmi **15 000+ codes possibles**
2. **Génération de documents** : Remplir des formulaires officiels (DAU, D10) avec précision
3. **Risque d'erreur** : Une mauvaise classification = amendes, retards, blocages en douane

### Constat

- Les transitaires passent **des heures** à chercher le bon code
- Les documents sont souvent remplis manuellement, source d'erreurs
- Aucun outil unifié n'existe pour le flux France-Algérie

## Notre Solution

Une **plateforme web intelligente** qui automatise les procédures douanières :

```
┌──────────────────┐     ┌──────────────────┐     ┌──────────────────┐
│     ÉTAPE 1      │────▶│     ÉTAPE 2      │────▶│     ÉTAPE 3      │
│  Classification  │     │    Documents     │     │   Transmission   │
│     TARIC        │     │    DAU / D10     │     │   (Future)       │
│                  │     │                  │     │                  │
│  🤖 Chatbot IA   │     │  📄 PDF auto     │     │  🔄 DELTA/BADR   │
└──────────────────┘     └──────────────────┘     └──────────────────┘
        ✅                       ✅                       🔄
```

**Valeur ajoutée** : L'utilisateur upload une photo de son produit, discute avec un chatbot, et obtient ses documents officiels prêts à l'emploi.

---
# 2. Architecture Générale
---

## Stack Technique

| Couche | Technologies | Justification |
|--------|--------------|---------------|
| **Backend** | Django 5.x, Python 3.12 | Framework robuste, ORM puissant, écosystème mature |
| **IA** | OpenAI GPT-4o, Function Calling | Meilleur rapport qualité/prix pour le tool-calling |
| **PDF** | WeasyPrint | Génération PDF depuis HTML/CSS, open-source |
| **Frontend** | Bootstrap 5, Alpine.js, HTMX | Léger, réactif, sans build complexe |

### Pourquoi ces choix ?

- **Django** : Permet de développer rapidement avec un ORM relationnel adapté à notre modèle de données complexe
- **OpenAI GPT-4o** : Seul modèle supportant à la fois Vision (analyse d'images) et Function Calling fiable
- **Alpine.js + HTMX** : Alternative légère à React/Vue, parfait pour une app Django traditionnelle

## Modèle de Données

```
┌──────────┐
│   User   │
└────┬─────┘
     │ 1:N
     ▼
┌──────────────┐
│  Expedition  │ ◄── Une expédition = un dossier d'import/export
│  - reference │
│  - direction │     (FR→DZ ou DZ→FR)
└──────┬───────┘
       │ 1:5 (une étape par phase)
       ▼
┌────────────────┐
│ ExpeditionEtape│ ◄── Étapes : Classification, Documents, Transmission...
│  - numero      │
│  - status      │
└───────┬────────┘
        │
   ┌────┴────┬────────────┐
   │         │            │
   ▼         ▼            ▼
┌──────┐ ┌──────────┐ ┌────────────┐
│Class.│ │Documents │ │Expedition  │
│Data  │ │Data      │ │Document    │
│      │ │          │ │            │
│code  │ │consignee │ │fichier.pdf │
│TARIC │ │transport │ │photo.jpg   │
└──────┘ │valeurs...│ └────────────┘
         └──────────┘
```

**Principe clé** : Chaque étape a ses propres données (ClassificationData pour l'étape 1, DocumentsData pour l'étape 2) liées en 1:1.

---
# 3. ÉTAPE 1 : Classification TARIC par Chatbot IA
## (Cœur technique du projet)
---

## Qu'est-ce que le TARIC ?

Le **TARIC** (Tarif Intégré des Communautés Européennes) est le système de codification des marchandises de l'UE.

```
Exemple : Un ordinateur portable

Code TARIC : 8471 30 00 00
             ──── ── ── ──
              │   │  │  └── TARIC (spécificités UE)
              │   │  └───── NC (Nomenclature Combinée UE)
              │   └──────── Sous-position (OMC)
              └──────────── Chapitre 84 : Machines
```

**Difficulté** : Plus de 15 000 codes possibles, avec des subtilités (un verre à vin vs un verre à eau = codes différents).

## Notre Approche : Un Agent IA avec Outils

Plutôt qu'un simple chatbot question/réponse, nous avons créé un **agent autonome** capable d'utiliser des outils :

```
┌─────────────────────────────────────────────────────────────────┐
│                         AGENT TARIC                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   L'utilisateur demande : "Classifie ce produit"               │
│                              │                                  │
│                              ▼                                  │
│                     ┌─────────────────┐                         │
│                     │    GPT-4o       │                         │
│                     │  (Cerveau IA)   │                         │
│                     └────────┬────────┘                         │
│                              │                                  │
│          L'IA DÉCIDE quels outils utiliser                      │
│                              │                                  │
│     ┌────────────────────────┼────────────────────────┐        │
│     ▼                        ▼                        ▼        │
│ ┌─────────┐           ┌─────────────┐          ┌───────────┐   │
│ │  web    │           │   browse    │          │  analyze  │   │
│ │ search  │           │   webpage   │          │ documents │   │
│ │         │           │             │          │           │   │
│ │Recherche│           │ Lit les     │          │ Analyse   │   │
│ │Google   │           │ pages web   │          │ photos/PDF│   │
│ └─────────┘           └─────────────┘          └───────────┘   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Pourquoi cette approche ?

1. **Autonomie** : L'IA décide elle-même si elle a besoin de chercher sur internet ou d'analyser un document
2. **Information à jour** : Les codes TARIC changent chaque année, la recherche web donne les infos actuelles
3. **Analyse multimodale** : L'IA peut "voir" les photos du produit grâce à GPT-4o Vision

## Comment fonctionne le Function Calling ?

OpenAI permet de définir des "outils" que l'IA peut appeler. Le processus :

```
1. On envoie à l'API la liste des outils disponibles
   
2. L'IA analyse la demande et répond :
   "Je veux appeler l'outil 'web_search' avec query='code TARIC verre cristal'"
   
3. Notre code exécute vraiment la recherche Google
   
4. On renvoie le résultat à l'IA
   
5. L'IA peut :
   - Appeler un autre outil (boucle)
   - Répondre à l'utilisateur avec sa classification
```

**Avantage technique** : L'IA ne "hallucine" pas les résultats de recherche, elle utilise de vraies données.

## Les 4 Outils du Chatbot

### 1. `web_search` - Recherche Google

**Objectif** : Trouver des informations actualisées sur les codes TARIC.

**Fonctionnement** :
- Utilise l'API Google Custom Search
- Retourne les titres, URLs et extraits des résultats
- L'IA peut ensuite demander à lire une page spécifique

**Pourquoi ?** Les codes TARIC sont mis à jour annuellement. Un LLM entraîné en 2023 ne connaît pas les modifications de 2024.

### 2. `browse_webpage` - Lecture de Pages Web

**Objectif** : Extraire l'information pertinente d'une page web.

**Innovation : Extraction Progressive avec Early Stopping**

Le problème : Une page web peut contenir 50 000 caractères, mais l'info utile est peut-être dans les 500 premiers.

```
MÉTHODE CLASSIQUE :
└─▶ Envoyer 50 000 caractères au LLM
└─▶ Coût : ~12 500 tokens (≈ 0.04$)

NOTRE MÉTHODE (chunks de 1 250 caractères) :
└─▶ Chunk 1 : "Info trouvée ?" → "NON"
└─▶ Chunk 2 : "Info trouvée ?" → "NON"
└─▶ Chunk 3 : "Info trouvée ?" → "OUI, code 8471.30"
    ✓ STOP ! Seulement 3 750 caractères traités
    ✓ Économie : ~92% des tokens
```

**Pourquoi cette optimisation ?** En production, chaque token coûte de l'argent. Avec des centaines d'utilisateurs, l'économie devient significative.

### 3. `analyze_documents` - Analyse Vision

**Objectif** : Analyser les photos et fiches techniques du produit.

**Innovation : Stratégie Intelligente pour les PDFs**

Les PDFs peuvent être :
- **Textuels** : Le texte est extractible (avec PyPDF2) → Gratuit
- **Scannés** : Images sans texte extractible → Nécessite Vision (coûteux)

```
Notre stratégie :

1. Essayer d'extraire le texte du PDF (gratuit)
   │
   ├─▶ Texte suffisant ? → Analyser le texte directement
   │
   └─▶ Texte insuffisant ? → Convertir en image → GPT-4o Vision
```

**Pourquoi ?** GPT-4o Vision coûte ~0.003$ par image. Pour les PDFs textuels, on économise 100% du coût Vision.

### 4. `fetch_pdf` - Téléchargement de Documents

**Objectif** : Télécharger et sauvegarder les fiches techniques trouvées sur le web.

**Scénario typique** :
1. L'IA cherche "fiche technique produit X"
2. Elle trouve un PDF sur le site du fabricant
3. Elle appelle `fetch_pdf` pour le télécharger
4. Le PDF est sauvegardé dans l'expédition
5. Elle peut ensuite l'analyser avec `analyze_documents`

**Valeur ajoutée** : L'utilisateur n'a pas besoin de télécharger manuellement les documents, l'IA le fait pour lui.

## Extraction Structurée des Résultats

**Problème** : L'IA répond en texte libre. Comment extraire les codes TARIC de manière fiable ?

**Solution** : OpenAI Structured Output (mode strict)

On définit un schéma JSON que l'IA **doit** respecter :

```json
{
  "proposals": [
    {
      "code_taric": "8471300000",
      "description": "Ordinateurs portables",
      "probability": 95,
      "justification": "Basé sur les caractéristiques..."
    }
  ]
}
```

**Avantage** : 
- Garantie à 100% que le JSON est valide
- Pas besoin de parser du texte avec des regex fragiles
- Le frontend peut afficher directement les propositions comme boutons cliquables

---
# 4. ÉTAPE 2 : Génération de Documents Douaniers
---

## Documents Générés

| Direction | Document | Description |
|-----------|----------|-------------|
| France → Algérie | **DAU** | Document Administratif Unique (exportation UE) |
| Algérie → France | **D10** | Déclaration de mise à la consommation |

Ces documents sont **obligatoires** pour toute opération douanière.

## Processus de Génération

```
1. DONNÉES D'ENTRÉE
   │
   ├─ Expédition (référence, direction)
   ├─ ClassificationData (code TARIC de l'étape 1)
   └─ DocumentsData (destinataire, transport, valeurs...)
       │
       ▼
2. TEMPLATE HTML (Django)
   │
   └─ Formulaire officiel reproduit en HTML/CSS
       │
       ▼
3. WEASYPRINT
   │
   └─ Conversion HTML → PDF (format A4)
       │
       ▼
4. SAUVEGARDE
   │
   └─ ExpeditionDocument (fichier PDF stocké)
```

**Pourquoi WeasyPrint ?**
- Open-source (pas de licence)
- Supporte le CSS moderne
- Parfait pour reproduire des formulaires officiels

## Modèle DocumentsData

Plus de **50 champs** pour capturer toutes les informations commerciales :

| Catégorie | Exemples de champs |
|-----------|--------------------|
| **Destinataire** | Nom, adresse, ville, pays, numéro fiscal |
| **Transport** | Mode (mer/air/route), nom du navire, ports |
| **Valeurs** | FOB, fret, assurance, CIF (calculé auto) |
| **Marchandises** | Nombre de colis, poids brut/net, origine |
| **Incoterms** | EXW, FOB, CIF, DAP... |

**Calcul automatique** : La valeur CIF est calculée automatiquement (FOB + Fret + Assurance).

---
# 5. Frontend et Qualité du Code
---

## Technologies Frontend

| Technologie | Usage |
|-------------|-------|
| **Bootstrap 5** | Layout responsive, composants UI |
| **Alpine.js** | Réactivité légère (drag & drop, formulaires) |
| **HTMX** | Requêtes AJAX déclaratives, mise à jour partielle |
| **Marked.js** | Rendu Markdown dans le chat |

**Design** : Interface minimaliste, inspirée Apple, avec variables CSS personnalisées.

## Qualité du Code

| Aspect | Notre Implémentation |
|--------|----------------------|
| **Service Layer** | Logique métier isolée dans des services (`TARICClassificationService`, `DocumentGenerationService`) |
| **Registry Pattern** | Outils du chatbot enregistrés dynamiquement |
| **Type Hints** | Toutes les fonctions publiques sont typées |
| **Logging** | Logs structurés par module pour le debugging |
| **Gestion d'erreurs** | Exceptions capturées, messages utilisateur clairs |

---
# 6. Perspectives et Évolutions
---

## Étape 3 : Transmission Électronique (En Recherche)

| Système | Pays | Approche Envisagée |
|---------|------|--------------------|
| **DELTA** | France | API via prestataires agréés (Conex, CCS) |
| **BADR** | Algérie | Automatisation web (RPA) si autorisé légalement |

**Défi** : Les systèmes douaniers n'ont généralement pas d'API publique. L'intégration nécessite soit un partenariat, soit une automatisation web.

---
# Démonstration
---

**Scénario (3 min)** :

1. Créer une expédition (France → Algérie)
2. Uploader une photo du produit
3. Discuter avec le chatbot → Il propose des codes TARIC
4. Sélectionner le code approprié
5. Remplir les données commerciales
6. Générer et télécharger le DAU

**URL** : https://vocesalviento.info/

---
# Résumé
---

| Critère | Ce que nous avons démontré |
|---------|----------------------------|
| **Qualité du Code** | Architecture modulaire, patterns professionnels (Service Layer, Registry) |
| **Maîtrise Technique** | Agent IA avec outils, extraction progressive, Vision, Structured Output |
| **Frontend** | Interface moderne avec Bootstrap, Alpine.js, HTMX |
| **Cas d'Usage** | Workflow complet fonctionnel : Classification → Documents |

---

## Merci de votre attention

*Des questions ?*